---
title: Object Detection with Azure Custom Vision - Part 1  
author: "Francisco Mussari"  
date: 2022-10-08  
categories: [computer-vision, deeplearning, azure, custom-vision, object-detection]  

---

Building an Object Detection with Azure Custom Vision - Part 1

![](test-url-Iteration2.png)

## Introduction

I was introduced to deep learning with fastai's top-down approach. That means that in the first week I was able to train a model
As part of fast.ai great deep learning course I was able to easily train and put into production a classifier of telecommunication tower elements.

## References

- Microsoft Learn Excersice: [Detect Objects in Images with Custom Vision](https://microsoftlearning.github.io/AI-102-AIEngineer/Instructions/18-object-detection.html)
- Custom Vision Documentation: [Quickstart: Create an object detection project with the Custom Vision client library](https://docs.microsoft.com/en-us/azure/cognitive-services/custom-vision-service/quickstarts/object-detection?tabs=visual-studio&pivots=programming-language-python)
- REST API Endpoint: [Custom Vision REST API reference - Azure Cognitive Services](https://docs.microsoft.com/en-us/rest/api/custom-vision/)
- APIs Documentation: [Custom_Vision_Training_3.3](https://southcentralus.dev.cognitive.microsoft.com/docs/services/Custom_Vision_Training_3.3)
- Azure SDK for Python: [Azure Cognitive Services Computer Vision SDK for Python](https://docs.microsoft.com/en-us/python/api/overview/azure/cognitiveservices-vision-computervision-readme?view=azure-python)
- Source Code: [Azure/azure-sdk-for-python](https://github.com/Azure/azure-sdk-for-python/blob/main/sdk/cognitiveservices/azure-cognitiveservices-vision-customvision/azure/cognitiveservices/vision/customvision/training/operations/_custom_vision_training_client_operations.py)

## Install and Import Libraries

After installing Custom Vision's Python SDK and python-dotenv  
`! pip install azure-cognitiveservices-vision-customvision==3.1.0`  
`! pip install python-dotenv`

In [ ]:
from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.training.models import ImageFileCreateBatch, ImageFileCreateEntry, Region
from msrest.authentication import ApiKeyCredentials
import time
import json
import os

import pandas as pd
from pathlib import Path

from PIL import Image
from PIL import UnidentifiedImageError

In [ ]:
# Borrowed from fastai library
def verify_image(fn):
    "Confirm that `fn` can be opened"
    try:
        im = Image.open(fn)
        im.draft(im.mode, (32,32))
        im.load()
        return True
    except: return False
    #except PIL.UnidentifiedImageError:

In [ ]:
def Upload_Images_1by1(picture_names: list) -> list('dict'):
    """Upload the pictures from a list of paths,
    one by one to keep track of the relation between
    local image name and Azure image id.
    And to track the ones that python fails opening
    """
    print("Uploading images...")

    processed_ids = []
    processed_status = []

    for pic_name in picture_names:
        pic_path = pictures_path / pic_name

        if verify_image(pic_path):
            with open(pic_path, mode="rb") as image_data:
                image_entry = ImageFileCreateEntry(name=pic_name, contents=image_data.read())
            
            # Upload the list of (1) images as a batch
            upload_result = training_client.create_images_from_files(
                custom_vision_project.id, 
                # Creates an ImageFileCreateBatch from a list of (1) ImageFileCreateEntry
                ImageFileCreateBatch(images=[image_entry])
            )
            # Check for failure
            if not upload_result.is_batch_successful:
                pic_status = upload_result.images[0].status
                pic_id = None
            else:
                pic_status = upload_result.images[0].status
                pic_id = upload_result.images[0].image.id
        else:
            pic_status = "ReadError" # Equivalente to SDK `ErrorSource`
            pic_id = None
        
        processed_status.append(pic_status)
        processed_ids.append(pic_id)
        print(pic_name, "//", pic_id, "//", pic_status)
    
    return [{"image_name": image_name, "image_id": image_id, "image_status": image_status} 
            for image_name, image_id, image_status in 
            zip(picture_names, processed_ids, processed_status)]